# Librerías a utilizar

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# Análisis exploratorio de los datos

In [2]:
track_features = pd.read_csv('data/tf_mini.csv')
print('Dataset Track Features:')
print('- Filas:', track_features.shape[0])
print('- Columnas:', track_features.shape[1])
print('- Datos nulos:', track_features.isna().sum().sum())

Dataset Track Features:
- Filas: 50704
- Columnas: 30
- Datos nulos: 0


In [4]:
sessions = pd.read_csv('data/log_mini.csv')
print('Dataset Sessions:')
print('- Filas:', sessions.shape[0])
print('- Columnas:', sessions.shape[1])
print('- Datos nulos:', sessions.isna().sum().sum())


SyntaxError: EOL while scanning string literal (Temp/ipykernel_10632/2536414020.py, line 4)

# Algoritmos de recomendación

# Resultados preliminares